# Separate the test mixture

In [ ]:
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
import time
import os
import librosa
import numpy as np
import IPython.display as ipd

from hparams import create_hparams
from model import Model
from soundfile import write

#### Set parameters

In [ ]:
mix_file_path = 'path-to-mixture'
mix_file_name = 'mixture'
model_name = 'unet'
model_path = 'path-to-model'

hparams = create_hparams()
hparams.hard_mask = False
hparams.extract = True

outputs = ['drums', 'bass', 'other', 'vocals', 'accompaniment']

#### Separate the target

In [ ]:
model = Model(hparams)

mix, _ = librosa.load(mix_file_path, sr=hparams.SR)
print('Start separating...')
start = time.time()

output_wavs = model.separate(mix, hparams, model_path)
print('Separating finished.')
print('Song duration: %.5fs, time taken: %.5fs' % (mix.shape[0]/hparams.SR, time.time()-start))

#### Mixture & target

Mixture

In [ ]:
mix = librosa.resample(mix, hparams.SR, 44100)
ipd.Audio(mix, rate=44100)

Target

In [ ]:
target = 'accompaniment'
print(target)
target_wav = librosa.resample(output_wavs[outputs.index(target)], hparams.SR, 44100)
ipd.Audio(target_wav, rate=44100)

In [ ]:
target = 'vocals'
print(target)
target_wav = librosa.resample(output_wavs[outputs.index(target)], hparams.SR, 44100)
ipd.Audio(target_wav, rate=44100)

In [ ]:
target = 'drums'
print(target)
target_wav = librosa.resample(output_wavs[outputs.index(target)], hparams.SR, 44100)
ipd.Audio(target_wav, rate=44100)

#### Write separated waves in the files

In [ ]:
for idx, output in enumerate(outputs):
    path_dir = os.path.join(hparams.separate_path, mix_file_name.split('.')[0], model_name)
    os.makedirs(path_dir, exist_ok=True)
    write(os.path.join(path_dir, output+'.wav'), output_wavs[idx], hparams.SR)

#### Separator for separating lists of files

In [ ]:
# separate files in a dir
class Separator(object):

    def __init__(self, haprams, mix_dir, model, checkpoint):
        self.hparams = hparams
        self.test_dir = mix_dir
        self.model_name = model
        self.model_path = checkpoint
        self.audio_list = os.listdir(self.test_dir)
        self.model = Model(hparams)
        self.model.load_model(self.model_path)

    
    def sep(self, ):
        for fname in self.audio_list:
            self.separate(fname)

    def separate(self, fname):
        mix, _ = librosa.load(os.path.join(self.test_dir, fname), sr=self.hparams.SR)
        insts_wav = self.model.separate(mix, self.hparams, self.model_path)

        insts = ['drums', 'bass', 'other', 'vocals', 'accompaniment']
        for idx, inst in enumerate(insts):
            path_dir = os.path.join(self.hparams.separate_path, fname.split('.')[0], self.model_name)
            os.makedirs(path_dir, exist_ok=True)
            write(os.path.join(path_dir, inst+'.wav'), insts_wav[idx], self.hparams.SR)